In [61]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OrdinalEncoder

In [62]:
X = datasets.fetch_covtype().data[:5000]
y = datasets.fetch_covtype().target[:5000]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print(X_train.shape, y_train.shape)

enc = OrdinalEncoder()
y_train = enc.fit_transform(y_train.reshape(-1, 1)).reshape(-1, )
y_test = enc.transform(y_test.reshape(-1, 1)).reshape(-1, )
print(np.unique(y_train))  # 7分类任务

X_train, X_test, y_train, y_test = tf.constant(X_train), tf.constant(X_test), tf.constant(y_train), tf.constant(y_test)

(4500, 54) (4500,)
[0. 1. 2. 3. 4. 5. 6.]


In [63]:
inputs = tf.keras.Input(shape=(54,))
x = tf.keras.layers.Dense(256, activation='tanh')(inputs)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(128)(x)
predictions = tf.keras.layers.Dense(7)(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)

model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 54)]              0         
                                                                 
 dense_40 (Dense)            (None, 256)               14080     
                                                                 
 dropout_10 (Dropout)        (None, 256)               0         
                                                                 
 dense_41 (Dense)            (None, 128)               32896     
                                                                 
 dense_42 (Dense)            (None, 128)               16512     
                                                                 
 dense_43 (Dense)            (None, 7)                 903       
                                                                 
Total params: 64,391
Trainable params: 64,391
Non-trainabl

In [64]:
# Configures the model for training.
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # 优化器
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  # 损失函数
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])  # 评估函数(根据模型的输入与输出选择)

In [65]:
# Trains the model for a fixed number of epochs (iterations on a dataset).
model.fit(X_train,  # Input data
          y_train,  # Target data
          # x是生成器或 tf.data.Dataset 的对象时,将忽略此参数
          # Boolean (whether to shuffle the training data before each epoch) or str (for 'batch'). This argument is ignored when x is a generator or an object of tf.data.
          shuffle=True,
          # Integer or None. Number of samples per gradient update. If unspecified, batch_size will default to 32.
          batch_size=64,
          epochs=10,  # Integer. Number of epochs to train the model.
          # 'auto', 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. 'auto' defaults to 1 for most cases
          verbose=2,
          # Data on which to evaluate the loss and any model metrics at the end of each epoch. The model will not be trained on this data.
          validation_data=(X_test, y_test),
          # Float between 0 and 1. Fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch.
          # validation_split=0.2 # 验证数据集占训练数据集的比例,取值范围为0~1
          )

Epoch 1/10
71/71 - 1s - loss: 1.4232 - sparse_categorical_accuracy: 0.4276 - val_loss: 1.2295 - val_sparse_categorical_accuracy: 0.4940 - 558ms/epoch - 8ms/step
Epoch 2/10
71/71 - 0s - loss: 1.2620 - sparse_categorical_accuracy: 0.4864 - val_loss: 1.1793 - val_sparse_categorical_accuracy: 0.5280 - 186ms/epoch - 3ms/step
Epoch 3/10
71/71 - 0s - loss: 1.2100 - sparse_categorical_accuracy: 0.5131 - val_loss: 1.1325 - val_sparse_categorical_accuracy: 0.5180 - 200ms/epoch - 3ms/step
Epoch 4/10
71/71 - 0s - loss: 1.1626 - sparse_categorical_accuracy: 0.5309 - val_loss: 1.1076 - val_sparse_categorical_accuracy: 0.5400 - 185ms/epoch - 3ms/step
Epoch 5/10
71/71 - 0s - loss: 1.1721 - sparse_categorical_accuracy: 0.5258 - val_loss: 1.0949 - val_sparse_categorical_accuracy: 0.5420 - 178ms/epoch - 3ms/step
Epoch 6/10
71/71 - 0s - loss: 1.1304 - sparse_categorical_accuracy: 0.5509 - val_loss: 1.0657 - val_sparse_categorical_accuracy: 0.5480 - 179ms/epoch - 3ms/step
Epoch 7/10
71/71 - 0s - loss: 1.12

In [66]:
model.history.history.items()

dict_items([('loss', [1.4232184886932373, 1.2620054483413696, 1.209990382194519, 1.1626187562942505, 1.172075867652893, 1.1303749084472656, 1.1248104572296143, 1.1090272665023804, 1.0671584606170654, 1.1208525896072388]), ('sparse_categorical_accuracy', [0.42755556106567383, 0.4864444434642792, 0.5131111145019531, 0.5308889150619507, 0.5257777571678162, 0.5508888959884644, 0.5382221937179565, 0.5531111359596252, 0.5671111345291138, 0.5397777557373047]), ('val_loss', [1.2295498847961426, 1.179253339767456, 1.1325305700302124, 1.1075630187988281, 1.094923734664917, 1.0657413005828857, 1.053030014038086, 1.0130200386047363, 1.0785350799560547, 1.070927619934082]), ('val_sparse_categorical_accuracy', [0.49399998784065247, 0.527999997138977, 0.5180000066757202, 0.5400000214576721, 0.5419999957084656, 0.5479999780654907, 0.5559999942779541, 0.5740000009536743, 0.5580000281333923, 0.5460000038146973])])

In [67]:
# Returns the loss value & metrics values for the model in test mode.
'''
batch_size:Integer or None. Number of samples per batch of computation. If unspecified, batch_size will default to 32. Do not specify the batch_size if your data is in the form of a dataset, generators, or keras.utils.Sequence instances (since they generate batches).
verbose:0 or 1. Verbosity mode. 0 = silent, 1 = progress bar.
'''
loss, metric = model.evaluate(X_test, y_test, batch_size=32, verbose=0)

print("test loss:", loss)
print("test acc:", metric)

test loss: 1.070927619934082
test acc: 0.5460000038146973


In [68]:
# Generates output predictions for the input samples.
'''
batch_size:Integer or None. Number of samples per batch. If unspecified, batch_size will default to 32. Do not specify the batch_size if your data is in the form of dataset, generators, or keras.utils.Sequence instances (since they generate batches).
verbose:Verbosity mode, 0 or 1.
'''
predictions = model.predict(X_test[:10],  # 预测10条样本
                            batch_size=32, verbose=0)
print('predictions shape:', predictions)

predictions shape: [[-1.1223283  -0.11800709  0.9677631   3.5925333   1.7195494   1.3187754
  -4.568822  ]
 [-1.5133486  -0.04070579  1.2694695   2.8417776   1.5223855   1.6578128
  -4.661986  ]
 [ 0.03818861  3.1082284  -2.4105046  -2.596511    8.074043   -0.9691935
  -3.7565541 ]
 [ 4.8502693   5.601515   -4.8249006  -4.016908    1.7941378  -4.814265
   0.26392996]
 [-0.80915546  0.7353264   0.4899541   2.7188685   0.08082309  0.9276882
  -4.697406  ]
 [-3.4095166  -0.66610247  3.6717043   2.869455    2.0083973   2.7781773
  -6.022371  ]
 [ 2.475797    2.521732   -3.3034635  -2.0285652   0.27793413 -2.165133
   2.3084633 ]
 [-0.92090034  0.21772142  0.9033706   2.197664    1.6636268   1.2595314
  -3.9238727 ]
 [-0.57327616  1.0557333   0.46287307  3.0685923   0.14112385  0.7818373
  -4.731491  ]
 [-0.47078878  0.52693015  0.09100114  2.5435739   1.3250715   0.80692154
  -3.128163  ]]
